### Zadanie 1 - Paradoks Simpsona##

Paradoks Simpsona mówi o tym, że badając pewne zjawisko możemy otrzymywać inne wyniki w przypadku rozpatrywania badanych grup odzielnie a inne, często odwrotne, jeśli dokonamy połączenia badanych grup.

#### Przykład:  Psy czy koty? ####

Chcemy sprawdzić które ze zwierzaków są bardziej popularne, więc przepytujemy znajomych. Wśród nich mamy zarówno kobiety jak i mężczyzn, więc dzielimy wyniki naszego wywiadu na dwie grupy. Okazuje się, że wśród naszych kolegów psy cieszą się dużo większą popularnością - aż 80%, podczas gdy tylko 40% koleżanek preferuje psy. Tak więc koty lubi zaledwie 20% naszych przyjaciół oraz 60% przyjaciółek. Wydawać by się mogło, że większość naszych przyjaciół wolałaby mieć psa. Jednak po podsumowaniu wyników okazuje się, że podział na fanów kotów i psów wśród znajomych jest prawie 50:50.

In [29]:
import pandas as pd
import numpy as np
col = ['M', 'K', 'Razem']
zw = ['Psy', 'Koty']
df = pd.DataFrame(np.array(['80%', '40%', '48%' ,'20%', '60%', '52']).reshape((2,3)), columns=col, index = zw)
df

,M,K,Razem
Psy,80%,40%,48%
Koty,20%,60%,52


Skąd taka różnica? Wynika z różnej liczebności obu grup: kolegów mamy łącznie 5 a koleżanek aż 20.

In [30]:
df2 = pd.DataFrame(np.array([4, 8, 12, 1, 12, 13]).reshape((2,3)), columns=col, index=zw)
df2

,M,K,Razem
Psy,4,8,12
Koty,1,12,13


#### Wpływ na metody pomiaru kampanii marketingowych ####

Wystarczy, że w powyższym przykładzie zastąpimy Psy i Koty dwiema kampaniami marketingowymi przeprowadzanymi na różnych grupach docelowych - chociażby na mężczyznach i kobietach. Jeśli uznamy, że na pierwszą kampanię marketingową odpowiedziało 80% mężczyzn i 40% kobiet a na drugą 20% mężczyzn i 60% kobiet, bez podawania liczebności każdej z płci możemy otrzymać błędne oszacowanie skuteczności kampanii - które w rzeczywistości są prawie tak samo skuteczne.

### Zadanie 2 - Raport skuteczności kampanii marketingowej ###

1. Stworzenie zbioru składającego się z klientów, którzy zrobili zakupy na Allegro w ciągu tych dwóch miesięcy wraz z sumaryczną kwotą wydatków w tym okresie:
Wybranie tych dla których ID_Sklepu = alg oraz Data_Trx Between(' 


2. 

2. Podział na grupę kontrolną 